In [1]:
import * as bitcoin from "bitcoinjs-lib"
import { ECPair, networks, Psbt } from "bitcoinjs-lib"
import { RegtestUtils } from "regtest-client"

In [2]:
const client = new RegtestUtils({APIURL: "http://localhost:18222/1"})

In [3]:
async function createPayment(wif: string) {
    const keypairs = ECPair.fromWIF(wif, networks.regtest)
    const p2pkh = bitcoin.payments.p2pkh({pubkey: keypairs.publicKey, network: networks.regtest})
    return {
        payment: p2pkh,
        keys: keypairs
    }
}

In [4]:
function getWitnessUtxo(out: any): any {
  delete out.address;
  out.script = Buffer.from(out.script, 'hex');
  return out;
}

async function createInput(utils: RegtestUtils, sender: any, amount: number) {
    const unspent = await utils.faucetComplex(sender.output, amount)
    const utx = await utils.fetch(unspent.txId);
    const nonWitnessUtxo = Buffer.from(utx.txHex, 'hex');
    return {
     hash: unspent.txId,
     index: unspent.vout,
     nonWitnessUtxo
    };
}

In [8]:
const requester = await createPayment("93Dt9qVXNDyMcLR9Wq5QRS72pUtG1eQ1CtFLd36kM2qntvThQE3")

In [9]:
const input = await createInput(client, requester.payment, 2e6)
const data = bitcoin.payments.embed({data: [ Buffer.from("hello, world", "utf-8")]})

In [11]:
const tx = new Psbt({ network: networks.regtest});
tx.setVersion(2)
    .setLocktime(0)
    .addInput(input)
    .addOutput({
        address: "mhNzyD5ZdQZ38WSdRftnzE16msN96fmXpe",
        value: 2e4
    })
    .addOutput({
        script: data.output!,
        value: 0
    })
    .signAllInputs(requester.keys).validateSignaturesOfAllInputs();

true


In [12]:
const raw_tx = await tx.finalizeAllInputs().extractTransaction(true).toHex();

In [13]:
raw_tx

0200000001b7edeeb0d5c994bd3a564764d3ad4cf379c4a2b2192cf0005f83d8de58ffa508000000008b48304502210083a1b4666713abd86391a2a0b6b991931be7b7823243f6528bba6977cafbd19102204304a047198566738037e6d9f44000a759024c0d564ffbb24a51c8aa470f14f9014104c226c854246153a52226ea043de19a32f2e40bdb8c7b347381d527a96eefa3dcbfdc874c6f4e83224320477b898d332b4d990a62d927163631420765ef448accffffffff02204e0000000000001976a914146f282c2297b9f5dcfe66897eccc74d29eed56188ac00000000000000000e6a0c68656c6c6f2c20776f726c6400000000


In [14]:
await client.broadcast(raw_tx)

Transaction: [Typical Script](https://learnmeabitcoin.com/technical/scriptPubKey)

In [16]:
await client.height()

432


In [17]:
await client.mine(1)

[ '25a0d0a186ec94980e6185f50a6132cdf52449fbe0436c48aea2b690f9dfc326' ]


In [21]:
await client.unspents("mhNzyD5ZdQZ38WSdRftnzE16msN96fmXpe")

[
  {
    txId: '986136b5468c02cf44e2e50afc6fc2b7b5c5c169f6ae82cec652f258e5dde4f1',
    vout: 0,
    height: 433,
    value: 20000
  }
]
